In [1]:
import pandas as pd
import numpy as np
from typing import Any, Optional
import logging
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
heat = pd.read_csv("C:/Users/USER/OneDrive/바탕 화면/내 자료/4-1준비/체크난방/train_heat.csv", index_col=0)
heat.head()

# 컬럼명에서 train_heat. 제거
heat.columns = heat.columns.str.replace("train_heat.", "", regex=False)
heat.head(2)

# 날짜 데이터를 데이트타임으로 변경
heat['tm'] = pd.to_datetime(heat['tm'].astype(str), format="%Y%m%d%H")

# 263069개의 결측치 -> na로 대체
heat = heat.replace(-99, np.nan)

# 풍향 -9.9 값을 NaN으로 변경
heat['wd'] = heat['wd'].replace(-9.9, np.nan)

#타겟칼럼이 null이면 삭제
heat = heat.dropna(subset=['heat_demand'])
heat

,tm,branch_id,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,heat_demand
1,2021-01-01 01:00:00,A,-10.1,78.3,0.5,0.0,0.0,68.2,NaN,-8.2,281.0
2,2021-01-01 02:00:00,A,-10.2,71.9,0.6,0.0,0.0,69.9,NaN,-8.6,262.0
3,2021-01-01 03:00:00,A,-10.0,360.0,0.0,0.0,0.0,69.2,NaN,-8.8,266.0
4,2021-01-01 04:00:00,A,-9.3,155.9,0.5,0.0,0.0,65.0,NaN,-8.9,285.0
5,2021-01-01 05:00:00,A,-9.0,74.3,1.9,0.0,0.0,63.5,NaN,-9.2,283.0
...,...,...,...,...,...,...,...,...,...,...,...
499297,2023-12-31 19:00:00,S,3.2,233.5,0.4,2.5,0.0,91.5,NaN,2.8,34.0
499298,2023-12-31 20:00:00,S,2.9,227.4,0.1,2.5,0.0,92.1,NaN,2.7,35.0
499299,2023-12-31 21:00:00,S,2.1,360.0,0.0,2.5,0.0,93.3,NaN,1.4,35.0
499300,2023-12-31 22:00:00,S,2.2,30.0,1.4,2.5,0.0,95.5,NaN,1.3,40.0


In [3]:
cluster_0 = heat[heat['branch_id'].isin(['E', 'F', 'I', 'J', 'K', 'N', 'O', 'Q'])].copy()
cluster_1 = heat[heat['branch_id'].isin(['B', 'C', 'G'])].copy()
cluster_2 = heat[heat['branch_id'].isin(['A', 'D', 'H', 'P'])].copy()
cluster_3 = heat[heat['branch_id'].isin(['L', 'M', 'R', 'S'])].copy()


In [12]:
cluster_0

,tm,branch_id,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,heat_demand
105117,2021-01-01 01:00:00,E,-7.0,282.3,0.9,0.0,0.0,NaN,NaN,-8.1,115.0
105118,2021-01-01 02:00:00,E,-8.4,152.9,0.7,0.0,0.0,NaN,NaN,-9.2,109.0
105119,2021-01-01 03:00:00,E,-8.9,167.2,0.4,0.0,0.0,NaN,NaN,-9.8,109.0
105120,2021-01-01 04:00:00,E,-8.5,271.5,0.4,0.0,0.0,NaN,NaN,-9.8,109.0
105121,2021-01-01 05:00:00,E,-9.7,189.6,0.4,0.0,0.0,NaN,NaN,-10.4,111.0
...,...,...,...,...,...,...,...,...,...,...,...
446739,2023-12-31 19:00:00,Q,0.6,360.0,0.0,7.0,0.0,95.4,NaN,0.1,143.0
446740,2023-12-31 20:00:00,Q,-0.2,278.1,0.8,7.0,0.0,96.0,NaN,-0.5,150.0
446741,2023-12-31 21:00:00,Q,-0.9,261.7,1.0,7.0,0.0,97.6,NaN,-1.2,159.0
446742,2023-12-31 22:00:00,Q,-1.9,88.8,0.2,7.0,0.0,98.2,NaN,-2.0,145.0


['E', 'F', 'I', 'J', 'K', 'N', 'O', 'Q']

In [4]:
def compute_null_streaks(df : pd.DataFrame, column_name) -> pd.DataFrame:
        """
        특정 column에 대해 branch별 연속 결측 구간을 계산하는 함수
        """
        result = []

        for branch in df['branch_id'].unique():
            df_branch = df[df['branch_id'] == branch].copy()

            # 결측 여부 마스크
            null_mask = df_branch[column_name].isnull()

            # 연속 구간 그룹 번호 부여
            group = (null_mask != null_mask.shift()).cumsum()

            # 결측만 필터링
            df_null = df_branch[null_mask].copy()
            df_null['group'] = group[null_mask]

            # 그룹별 통계
            summary = df_null.groupby('group').agg({
                'tm': ['count', 'min', 'max']
            }).reset_index()
            summary.columns = ['group', 'length', 'start_time', 'end_time']
            summary['branch_id'] = branch
            summary['feature'] = column_name
            summary['duration_days'] = (summary['end_time'] - summary['start_time']).dt.days + 1
            summary = summary[summary['length'] > 24]  # 장기 결측만
            result.append(summary)

        return pd.concat(result, ignore_index=True)
        
def fill_long_term_missing_with_cluster_avg(df: pd.DataFrame, summary_df: pd.DataFrame, variable) -> pd.DataFrame:
        '''장시간 결측 채우기'''

        for idx, row in summary_df.iterrows():
            branch = row['branch_id']
            start_time = row['start_time']
            end_time = row['end_time']

            print(f"[{idx+1}/{len(summary_df)}] 처리 중: branch = {branch}, 기간 = {start_time} ~ {end_time}")

            # 결측 구간에 해당하는 tm 범위
            mask_time = (df['tm'] >= start_time) & (df['tm'] <= end_time)
            
            # 결측인 행들 (해당 branch, 해당 변수에서 null)
            mask_missing = mask_time & (df['branch_id'] == branch) & (df[variable].isnull())
            
            # 같은 시간대, 같은 클러스터에서 결측이 아닌 값들의 평균 구하기
            avg_values = []
            for t in df.loc[mask_time, 'tm'].unique():
                mask_same_time = (df['tm'] == t) & (df[variable].notnull()) & (df['branch_id'] != branch)
                mean_val = df.loc[mask_same_time, variable].mean()
                avg_values.append((t, mean_val))
            
            # 평균값으로 채우기
            for t, val in avg_values:
                if pd.notnull(val):
                    df.loc[mask_missing & (df['tm'] == t), variable] = val

            print("결측 구간 처리 완료.")
        
        return df

def apply_mice(df, vars_to_impute):

        imputer = IterativeImputer(random_state=42)

        df_impute = df[vars_to_impute]
        
        imputed_array = imputer.fit_transform(df_impute)

        df_imputed = df.copy()

        # 보간된 컬럼명 뒤에 '_imputed' 붙여서 새 컬럼 생성
        for i, col in enumerate(vars_to_impute):
            df_imputed[col] = imputed_array[:, i]

        return df_imputed


def fill_si_by_pattern(df: pd.DataFrame, branches: set, patterns: list):
        '''해당 패턴일 때 & si변수가 null값이면 0으로 만드는 함수'''
        # 날짜+시간 추출

        df['month'] = df['tm'].dt.month
        df['day'] = df['tm'].dt.day
        df['hour'] = df['tm'].dt.hour

        mask = pd.Series(False, index=df.index)

        for (start_md, end_md, night_start, night_end) in patterns:
            start_m, start_d = start_md
            end_m, end_d = end_md

            if start_md <= end_md:
                date_mask = (
                    ((df['month'] > start_m) | ((df['month'] == start_m) & (df['day'] >= start_d))) &
                    ((df['month'] < end_m) | ((df['month'] == end_m) & (df['day'] <= end_d)))
                )
            else:
                # 연말-연초 넘는 경우
                date_mask = (
                    ((df['month'] > start_m) | ((df['month'] == start_m) & (df['day'] >= start_d))) |
                    ((df['month'] < end_m) | ((df['month'] == end_m) & (df['day'] <= end_d)))
                )

            # 시간 조건
            if night_start < night_end:
                time_mask = (df['hour'] >= night_start) & (df['hour'] < night_end)
            else:
                time_mask = (df['hour'] >= night_start) | (df['hour'] < night_end)

            mask |= date_mask & time_mask

        # 조건 만족시 si = 0
        df.loc[df['branch_id'].isin(branches) & df['si'].isna() & mask, 'si'] = 0

        # 정리
        return df

In [5]:
pattern_group_1 = [
            ((1, 1), (2, 6), 19, 7),
            ((2, 6), (3, 4), 20, 7),
            ((3, 4), (4, 7), 20, 6),
            ((4, 7), (4, 13), 21, 6),
            ((4, 13), (8, 28), 21, 5),
            ((8, 28), (9, 3), 21, 6),
            ((9, 3), (10, 13), 20, 6),
            ((10, 13), (11, 3), 19, 6),
            ((11, 3), (12, 31), 19, 7),
        ]

        # D/E/G/H/I/J/K/S용 패턴
pattern_group_2 = [
            ((1, 1), (2, 5), 19, 7),
            ((2, 5), (3, 3), 20, 7),
            ((3, 3), (4, 7), 20, 6),
            ((4, 7), (4, 13), 21, 6),
            ((4, 13), (8, 28), 21, 5),
            ((8, 28), (9, 3), 21, 6),
            ((9, 3), (10, 13), 20, 6),
            ((10, 13), (11, 4), 19, 6),
            ((11, 4), (12, 31), 19, 7),
        ]

        #L,M,N (유사)
pattern_group_3 = [
            ((1, 1),   (2, 10), 19, 7),  # 1월 1일 ~ 2월 10일 07시까지
            ((2, 10),  (2, 25), 20, 7),  # 2월 10일 ~ 2월 25일 07시까지
            ((2, 25),  (4, 9),  20, 6),  # 2월 25일 ~ 4월 9일 06시까지
            ((4, 9),   (4, 19), 20, 5),  # 4월 9일 ~ 4월 19일 05시까지
            ((4, 19),  (8, 26), 21, 5),  # 4월 19일 ~ 8월 26일 05시까지
            ((8, 26),  (9, 4),  20, 5),  # 8월 26일 ~ 9월 4일 05시까지
            ((9, 4),   (10, 7), 20, 6),  # 9월 4일 ~ 10월 7일 06시까지
            ((10, 7),  (11, 13), 19, 6), # 10월 7일 ~ 11월 13일 06시까지
            ((11, 13), (12, 31), 19, 7), # 11월 13일 ~ 12월 31일 07시까지
        ]

        #R
pattern_group_R = [
            ((1, 1),   (1, 31), 19, 7),  # 1월 1일 ~ 1월 31일 07시까지
            ((1, 31),  (3, 2),  20, 7),  # 1월 31일 ~ 3월 2일 07시까지
            ((3, 2),   (4, 9),  20, 6),  # 3월 2일 ~ 4월 9일 06시까지
            ((4, 9),   (4, 15), 21, 6),  # 4월 9일 ~ 4월 15일 06시까지
            ((4, 15),  (8, 26), 21, 5),  # 4월 15일 ~ 8월 26일 05시까지
            ((8, 26),  (9, 2),  21, 6),  # 8월 26일 ~ 9월 2일 06시까지
            ((9, 2),   (10, 14), 20, 6), # 9월 2일 ~ 10월 14일 06시까지
            ((10, 14), (11, 8), 19, 6),  # 10월 14일 ~ 11월 8일 06시까지
            ((11, 8),  (2, 2),  19, 7),  # 11월 8일 ~ 다음해 2월 2일 07시까지
        ]

        #O
pattern_group_O = [
            ((1, 1),   (2, 6),  19, 7),  # 1월 1일 ~ 2월 6일 07시까지
            ((2, 6),   (3, 2),  20, 7),  # 2월 6일 ~ 3월 2일 07시까지
            ((3, 2),   (4, 11), 20, 6),  # 3월 2일 ~ 4월 11일 06시까지
            ((4, 11),  (4, 13), 21, 6),  # 4월 11일 ~ 4월 13일 06시까지
            ((4, 13),  (8, 30), 21, 5),  # 4월 13일 ~ 8월 30일 05시까지
            ((8, 30),  (9, 1),  21, 6),  # 8월 30일 ~ 9월 1일 06시까지
            ((9, 1),   (10, 11), 20, 6), # 9월 1일 ~ 10월 11일 06시까지
            ((10, 11), (11, 7), 19, 6),  # 10월 11일 ~ 11월 7일 06시까지
            ((11, 7),  (12, 31), 19, 7), # 11월 7일 ~ 12월 31일까지
        ]

        #P
pattern_group_P = [
            ((1, 20),  (2, 6),  19, 7),  # 1월 20일 ~ 2월 6일 07시까지
            ((2, 6),   (3, 3),  20, 7),  # 2월 6일 ~ 3월 3일 07시까지
            ((3, 3),   (4, 10), 20, 6),  # 3월 3일 ~ 4월 10일 06시까지
            ((4, 10),  (4, 13), 21, 6),  # 4월 10일 ~ 4월 13일 06시까지
            ((4, 13),  (8, 30), 21, 5),  # 4월 13일 ~ 8월 30일 05시까지
            ((8, 30),  (9, 2),  21, 6),  # 8월 30일 ~ 9월 2일 06시까지
            ((9, 2),   (10, 12), 20, 6), # 9월 2일 ~ 10월 12일 06시까지
            ((10, 12), (11, 6), 19, 6),  # 10월 12일 ~ 11월 6일 06시까지
            ((11, 6),  (12, 31), 19, 7), # 11월 6일 ~ 12월 31일까지
        ]

        #Q
pattern_group_Q = [
            ((1, 1), (2, 5), 19, 7),
            ((2, 5), (3, 5), 20, 7),
            ((3, 5), (4, 7), 20, 6),
            ((4, 7), (4, 14), 21, 6),
            ((4, 14), (8, 28), 21, 5),
            ((8, 28), (9, 3), 21, 6),
            ((9, 3), (10, 13), 20, 6),
            ((10, 13), (11, 2), 19, 6),
            ((11, 2), (12, 31), 19, 7)
        ]

        # 브랜치와 패턴 그룹 매핑
branch_group_map = {
            'A': pattern_group_1,
            'B': pattern_group_1,
            'C': pattern_group_1,
            'F': pattern_group_1,
            'D': pattern_group_2,
            'E': pattern_group_2,
            'G': pattern_group_2,
            'H': pattern_group_2,
            'I': pattern_group_2,
            'J': pattern_group_2,
            'K': pattern_group_2,
            'S': pattern_group_2,
            'L': pattern_group_3,
            'M': pattern_group_3,
            'N': pattern_group_3,
            'O': pattern_group_O,
            'P': pattern_group_P,
            'Q': pattern_group_Q,
            'R': pattern_group_R,
        }


In [15]:
print(type(cluster_0))

<class 'pandas.core.frame.DataFrame'>


In [7]:

# cluster_0
# 반복 처리
for branches, pattern_group in branch_group_map.items():
    cluster_0 = fill_si_by_pattern(cluster_0, {branches}, pattern_group)


# 2. 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm' 변수
# 1)장시간 결측
vars_to_check = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si']

summaries = []
for var in vars_to_check:
    summary = compute_null_streaks(cluster_0, var)
    summaries.append(summary)

# 하나로 합치고, 중복 제거
full_summary = pd.concat(summaries, ignore_index=True)
unique_summary = full_summary.drop_duplicates(subset=['branch_id', 'start_time', 'end_time'])

for var in vars_to_check:
    fill_long_term_missing_with_cluster_avg(cluster_0, unique_summary, var)

[1/76] 처리 중: branch = E, 기간 = 2021-04-10 08:00:00 ~ 2021-04-12 16:00:00
결측 구간 처리 완료.
[2/76] 처리 중: branch = E, 기간 = 2021-05-22 08:00:00 ~ 2021-05-24 11:00:00
결측 구간 처리 완료.
[3/76] 처리 중: branch = E, 기간 = 2021-07-16 17:00:00 ~ 2021-07-19 15:00:00
결측 구간 처리 완료.
[4/76] 처리 중: branch = E, 기간 = 2021-07-21 18:00:00 ~ 2021-12-06 17:00:00
결측 구간 처리 완료.
[5/76] 처리 중: branch = E, 기간 = 2023-07-30 15:00:00 ~ 2023-07-31 16:00:00
결측 구간 처리 완료.
[6/76] 처리 중: branch = E, 기간 = 2023-08-16 16:00:00 ~ 2023-08-18 14:00:00
결측 구간 처리 완료.
[7/76] 처리 중: branch = F, 기간 = 2021-05-09 11:00:00 ~ 2021-05-10 17:00:00
결측 구간 처리 완료.
[8/76] 처리 중: branch = F, 기간 = 2022-08-09 23:00:00 ~ 2022-08-11 14:00:00
결측 구간 처리 완료.
[9/76] 처리 중: branch = F, 기간 = 2022-10-03 06:00:00 ~ 2022-10-04 13:00:00
결측 구간 처리 완료.
[10/76] 처리 중: branch = F, 기간 = 2022-11-10 04:00:00 ~ 2022-11-11 11:00:00
결측 구간 처리 완료.
[11/76] 처리 중: branch = F, 기간 = 2022-11-26 00:00:00 ~ 2022-11-27 10:00:00
결측 구간 처리 완료.
[12/76] 처리 중: branch = F, 기간 = 2023-06-19 08:00:00 ~ 2023-09-26

In [8]:
# cluster_1
# 반복 처리
for branches, pattern_group in branch_group_map.items():
    cluster_1 = fill_si_by_pattern(cluster_1, {branches}, pattern_group)

# 2. 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm' 변수
# 1)장시간 결측
vars_to_check = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si']

summaries = []
for var in vars_to_check:
    summary = compute_null_streaks(cluster_1, var)
    summaries.append(summary)

# 하나로 합치고, 중복 제거
full_summary = pd.concat(summaries, ignore_index=True)
unique_summary = full_summary.drop_duplicates(subset=['branch_id', 'start_time', 'end_time'])

for var in vars_to_check:
    fill_long_term_missing_with_cluster_avg(cluster_1, unique_summary, var)

[1/17] 처리 중: branch = B, 기간 = 2021-11-05 21:00:00 ~ 2021-11-08 09:00:00
결측 구간 처리 완료.
[2/17] 처리 중: branch = B, 기간 = 2022-11-14 09:00:00 ~ 2022-11-17 13:00:00
결측 구간 처리 완료.
[3/17] 처리 중: branch = B, 기간 = 2021-11-05 20:00:00 ~ 2021-11-08 09:00:00
결측 구간 처리 완료.
[5/17] 처리 중: branch = C, 기간 = 2022-07-23 01:00:00 ~ 2022-07-24 11:00:00
결측 구간 처리 완료.
[6/17] 처리 중: branch = G, 기간 = 2022-07-12 01:00:00 ~ 2022-07-14 13:00:00
결측 구간 처리 완료.
[7/17] 처리 중: branch = G, 기간 = 2022-07-15 06:00:00 ~ 2022-07-18 05:00:00
결측 구간 처리 완료.
[8/17] 처리 중: branch = G, 기간 = 2022-07-19 07:00:00 ~ 2022-07-20 23:00:00
결측 구간 처리 완료.
[9/17] 처리 중: branch = G, 기간 = 2022-07-23 07:00:00 ~ 2022-07-24 20:00:00
결측 구간 처리 완료.
[10/17] 처리 중: branch = G, 기간 = 2022-07-29 01:00:00 ~ 2022-08-07 03:00:00
결측 구간 처리 완료.
[11/17] 처리 중: branch = G, 기간 = 2022-08-07 06:00:00 ~ 2022-08-09 00:00:00
결측 구간 처리 완료.
[12/17] 처리 중: branch = G, 기간 = 2022-08-11 02:00:00 ~ 2022-08-12 23:00:00
결측 구간 처리 완료.
[13/17] 처리 중: branch = G, 기간 = 2022-08-14 07:00:00 ~ 2022-08-1

In [9]:
# cluster_2
# 반복 처리
for branches, pattern_group in branch_group_map.items():
    cluster_2 = fill_si_by_pattern(cluster_2, {branches}, pattern_group)


# 2. 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm' 변수
# 1)장시간 결측
vars_to_check = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si']

summaries = []
for var in vars_to_check:
    summary = compute_null_streaks(cluster_2, var)
    summaries.append(summary)

# 하나로 합치고, 중복 제거
full_summary = pd.concat(summaries, ignore_index=True)
unique_summary = full_summary.drop_duplicates(subset=['branch_id', 'start_time', 'end_time'])

for var in vars_to_check:
    fill_long_term_missing_with_cluster_avg(cluster_2, unique_summary, var)

[1/43] 처리 중: branch = D, 기간 = 2021-04-10 08:00:00 ~ 2021-04-12 16:00:00
결측 구간 처리 완료.
[2/43] 처리 중: branch = D, 기간 = 2021-05-22 08:00:00 ~ 2021-05-24 11:00:00
결측 구간 처리 완료.
[3/43] 처리 중: branch = D, 기간 = 2021-07-16 17:00:00 ~ 2021-07-19 15:00:00
결측 구간 처리 완료.
[4/43] 처리 중: branch = D, 기간 = 2021-07-21 18:00:00 ~ 2021-12-06 17:00:00
결측 구간 처리 완료.
[5/43] 처리 중: branch = D, 기간 = 2023-07-30 15:00:00 ~ 2023-07-31 16:00:00
결측 구간 처리 완료.
[6/43] 처리 중: branch = D, 기간 = 2023-08-16 16:00:00 ~ 2023-08-18 14:00:00
결측 구간 처리 완료.
[7/43] 처리 중: branch = D, 기간 = 2021-04-10 07:00:00 ~ 2021-04-12 16:00:00
결측 구간 처리 완료.
[10/43] 처리 중: branch = D, 기간 = 2021-07-21 17:00:00 ~ 2021-12-06 17:00:00
결측 구간 처리 완료.
[11/43] 처리 중: branch = D, 기간 = 2023-07-29 14:00:00 ~ 2023-07-31 17:00:00
결측 구간 처리 완료.
[12/43] 처리 중: branch = D, 기간 = 2023-08-16 15:00:00 ~ 2023-08-18 14:00:00
결측 구간 처리 완료.
[13/43] 처리 중: branch = D, 기간 = 2023-09-07 07:00:00 ~ 2023-09-10 00:00:00
결측 구간 처리 완료.
[14/43] 처리 중: branch = D, 기간 = 2023-09-11 01:00:00 ~ 2023-09-

In [10]:
# cluster_3
# 반복 처리
for branches, pattern_group in branch_group_map.items():
    cluster_3 = fill_si_by_pattern(cluster_3, {branches}, pattern_group)


# 2. 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm' 변수
# 1)장시간 결측
vars_to_check = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si']

summaries = []
for var in vars_to_check:
    summary = compute_null_streaks(cluster_3, var)
    summaries.append(summary)

# 하나로 합치고, 중복 제거
full_summary = pd.concat(summaries, ignore_index=True)
unique_summary = full_summary.drop_duplicates(subset=['branch_id', 'start_time', 'end_time'])

for var in vars_to_check:
    fill_long_term_missing_with_cluster_avg(cluster_3, unique_summary, var)

[1/11] 처리 중: branch = M, 기간 = 2022-03-30 22:00:00 ~ 2022-04-04 10:00:00
결측 구간 처리 완료.
[2/11] 처리 중: branch = S, 기간 = 2021-11-12 09:00:00 ~ 2021-11-17 16:00:00
결측 구간 처리 완료.
[3/11] 처리 중: branch = S, 기간 = 2021-11-17 18:00:00 ~ 2021-11-25 15:00:00
결측 구간 처리 완료.
[4/11] 처리 중: branch = S, 기간 = 2021-11-25 17:00:00 ~ 2021-11-30 16:00:00
결측 구간 처리 완료.
[5/11] 처리 중: branch = S, 기간 = 2022-01-12 22:00:00 ~ 2022-01-14 11:00:00
결측 구간 처리 완료.
[6/11] 처리 중: branch = S, 기간 = 2023-11-11 11:00:00 ~ 2023-11-12 14:00:00
결측 구간 처리 완료.
[8/11] 처리 중: branch = S, 기간 = 2021-11-12 09:00:00 ~ 2021-11-30 16:00:00
결측 구간 처리 완료.
[21/11] 처리 중: branch = M, 기간 = 2022-03-30 22:00:00 ~ 2022-04-04 11:00:00
결측 구간 처리 완료.
[22/11] 처리 중: branch = S, 기간 = 2021-11-12 09:00:00 ~ 2021-11-30 17:00:00
결측 구간 처리 완료.
[23/11] 처리 중: branch = S, 기간 = 2022-01-12 22:00:00 ~ 2022-01-14 12:00:00
결측 구간 처리 완료.
[24/11] 처리 중: branch = S, 기간 = 2023-11-11 11:00:00 ~ 2023-11-12 15:00:00
결측 구간 처리 완료.
[1/11] 처리 중: branch = M, 기간 = 2022-03-30 22:00:00 ~ 2022-04-0

In [11]:
# 각각 클러스터를 하나의 리스트로 모은 후 concat
merged_df = pd.concat([cluster_0, cluster_1, cluster_2, cluster_3], ignore_index=True)
merged_df.to_csv("C:/Users/USER/OneDrive/바탕 화면/내 자료/4-1준비/체크난방/장시간결측치만.csv")

## 단시간 결측은 따로!

In [12]:
long_df = pd.read_csv("C:/Users/USER/OneDrive/바탕 화면/내 자료/4-1준비/체크난방/장시간결측치만.csv")
long_df.head()

,Unnamed: 0,tm,branch_id,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,heat_demand,month,day,hour
0,0,2021-01-01 01:00:00,E,-7.0,282.3,0.9,0.0,0.0,76.000000,0.0,-8.1,115.0,1,1,1
1,1,2021-01-01 02:00:00,E,-8.4,152.9,0.7,0.0,0.0,75.433333,0.0,-9.2,109.0,1,1,2
2,2,2021-01-01 03:00:00,E,-8.9,167.2,0.4,0.0,0.0,77.133333,0.0,-9.8,109.0,1,1,3
3,3,2021-01-01 04:00:00,E,-8.5,271.5,0.4,0.0,0.0,78.000000,0.0,-9.8,109.0,1,1,4
4,4,2021-01-01 05:00:00,E,-9.7,189.6,0.4,0.0,0.0,79.800000,0.0,-10.4,111.0,1,1,5


In [13]:
long_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499278 entries, 0 to 499277
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   499278 non-null  int64  
 1   tm           499278 non-null  object 
 2   branch_id    499278 non-null  object 
 3   ta           497973 non-null  float64
 4   wd           495336 non-null  float64
 5   ws           495669 non-null  float64
 6   rn_day       493669 non-null  float64
 7   rn_hr1       493210 non-null  float64
 8   hm           497763 non-null  float64
 9   si           470000 non-null  float64
 10  ta_chi       499258 non-null  float64
 11  heat_demand  499278 non-null  float64
 12  month        499278 non-null  int64  
 13  day          499278 non-null  int64  
 14  hour         499278 non-null  int64  
dtypes: float64(9), int64(4), object(2)
memory usage: 57.1+ MB


In [ ]:
# 단시간결측 mice방법론
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def apply_mice(df, vars_to_impute):
    imputer = IterativeImputer(random_state=42)
    df_impute = df[vars_to_impute]
    imputed_array = imputer.fit_transform(df_impute)

    df_imputed = df.copy()

    # 보간된 컬럼명 뒤에 '_imputed' 붙여서 새 컬럼 생성
    for i, col in enumerate(vars_to_impute):
        df_imputed[col] = imputed_array[:, i]

    return df_imputed

# 변수 리스트 (결측값 있을만한 변수들)
vars_to_impute = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'ta_chi']
